In [ ]:
import torch
import os
import sys
import matplotlib.pyplot as plt
import argparse
import numpy as np
from tqdm.auto import tqdm
import torch.nn.functional as F

sys.path.append("../../")
from src.filepath import ABSOLUTE_PATH
from src.model.UNet2d import Unet2D
from src.model.fno import FNO2D
from src.utils.utils import plot_compare_2d, relative_error
from src.train.reaction_diffusion import cond_emb, renormalize
from src.train.reaction_diffusion import normalize_to_neg_one_to_one as normalize

In [2]:
model_type = "FNO"

In [ ]:
channel, out_dim, dim = 2, 1, 24
if model_type == "Unet":

    model1 = Unet2D(dim=dim, cond_emb=cond_emb(), out_dim=out_dim, dim_mults=(1, 2), channels=channel).to("cuda")

    model2 = Unet2D(dim=dim, cond_emb=cond_emb(), out_dim=out_dim, dim_mults=(1, 2), channels=channel).to("cuda")

    model1.load_state_dict(torch.load("../../results/reaction_diffusion/surrogateUnetu10000/model.pt")["model"])

    model2.load_state_dict(torch.load("../../results/reaction_diffusion/surrogateUnetv10000/model.pt")["model"])
elif model_type == "ViT":
    model1 = ViT(
        seq_len=20,
        patch_size=2,
        dim=64,
        depth=2,
        heads=8,
        mlp_dim=128,
        cond_emb=cond_emb(),
        dropout=0.0,
        emb_dropout=0.0,
        channels=11,
        out_channels=9,
    ).to("cuda")
    model2 = ViT(
        seq_len=20,
        patch_size=2,
        dim=64,
        depth=2,
        heads=8,
        mlp_dim=128,
        cond_emb=cond_emb(),
        dropout=0.0,
        emb_dropout=0.0,
        channels=11,
        out_channels=9,
    ).to("cuda")
    model1.load_state_dict(torch.load("../../results/reaction_diffusion/surrogateViTu10000/model.pt")["model"])

    model2.load_state_dict(torch.load("../../results/reaction_diffusion/surrogateViTv10000/model.pt")["model"])
elif model_type == "FNO":
    model1 = FNO2D(
        in_channels=channel,
        out_channels=out_dim,
        nr_fno_layers=4,
        fno_layer_size=24,
        fno_modes=[6, 12],
        time_input=False,
        cond_emb=cond_emb(),
    ).to("cuda")
    model2 = FNO2D(
        in_channels=channel,
        out_channels=out_dim,
        nr_fno_layers=4,
        fno_layer_size=24,
        fno_modes=[6, 12],
        time_input=False,
        cond_emb=cond_emb(),
    ).to("cuda")
    model1.load_state_dict(torch.load("../../results/reaction_diffusion/surrogateFNOu10000/model.pt")["model"])

    model2.load_state_dict(torch.load("../../results/reaction_diffusion/surrogateFNOv10000/model.pt")["model"])

validiation of u

In [ ]:
data = (
    torch.tensor(np.load("../../data/reaction_diffusion/reaction_diffusion_u_from_v_u.npy")).float().to("cuda")[9000:]
)
cond = (
    torch.tensor(np.load("../../data/reaction_diffusion/reaction_diffusion_u_from_v_v.npy")).float().to("cuda")[9000:]
)
data = torch.tensor(data).unsqueeze(1)
cond1 = torch.tensor(cond).unsqueeze(1)
cond2 = data[:, :, 0:1].clone().expand(-1, -1, data.shape[2], -1)
cond = torch.concat((cond1, cond2), dim=1)


with torch.no_grad():

    u_pred = renormalize(model1(normalize(cond)))


rmse = relative_error(data, u_pred)


rmse, F.mse_loss(data, u_pred)

In [ ]:
random_n = np.random.randint(0, data.shape[0])
plot_compare_2d(
    true_d=data[random_n, 0],
    pred_d=u_pred[random_n, 0],
)
# plot_compare_2d(data, un(u_pred))

validiation of v

In [ ]:
data = (
    torch.tensor(np.load("../../data/reaction_diffusion/reaction_diffusion_v_from_u_v.npy")).float().to("cuda")[9000:]
)
cond = (
    torch.tensor(np.load("../../data/reaction_diffusion/reaction_diffusion_v_from_u_u.npy")).float().to("cuda")[9000:]
)
data = torch.tensor(data).unsqueeze(1)
cond1 = torch.tensor(cond).unsqueeze(1)
cond2 = data[:, :, 0:1].clone().expand(-1, -1, data.shape[2], -1)
cond = torch.concat((cond1, cond2), dim=1)

with torch.no_grad():
    v_pred = renormalize(model2(normalize(cond)))


rmse = relative_error(data, v_pred)
rmse, F.mse_loss(data, v_pred)

In [ ]:
random_n = np.random.randint(0, data.shape[0])
plot_compare_2d(
    true_d=data[random_n, 0],
    pred_d=(v_pred)[random_n, 0],
)
# plot_compare_2d(data, un(u_pred))

compose

In [ ]:
data = (
    torch.tensor(np.load(ABSOLUTE_PATH + "/data/reaction_diffusion/reaction_diffusion_uv.npy"))
    .float()
    .to("cuda")[:2000]
)
# data = (data + 5) / 10

data = data.permute(0, 2, 1)

# data1 = np.load('../../data/reaction_diffusion_u_from_v_u.npy')

u, v = data[..., :20].unsqueeze(1), data[..., 20:].unsqueeze(1)

u_intial, v_intial = u[:, :, 0:1].expand(-1, -1, 10, -1), v[:, :, 0:1].expand(-1, -1, 10, -1)

u.shape, u_intial.shape

In [ ]:
u_intial_n, v_intial_n = normalize(u_intial), normalize(v_intial)
alpha = 0.5


with torch.no_grad():

    i = 0

    eps = 1

    v_iter = torch.ones_like(v)

    u_iter = torch.ones_like(u)

    while i < 100 and eps > 2e-5:

        i = i + 1

        condv = torch.concat((v_iter, u_intial_n), dim=1)

        u_iter_new = model1(condv)

        condu = torch.concat((u_iter, v_intial_n), dim=1)

        v_iter_new = model2(condu)

        eps = F.l1_loss(u_iter, u_iter_new) + F.l1_loss(v_iter, v_iter_new)

        u_iter = u_iter_new * (1 - alpha) + alpha * u_iter

        v_iter = v_iter_new * (1 - alpha) + alpha * v_iter

        print("iteration: ", i, " eps: ", eps)

In [ ]:
mult_p_true = torch.concat((u, v), dim=1)
mult_p_pred = renormalize(torch.concat((u_iter, v_iter), dim=1))


mult_p_true.shape, mult_p_pred.shape

In [ ]:
relative_error(mult_p_pred[:, 0], mult_p_true[:, 0]), relative_error(mult_p_pred[:, 1], mult_p_true[:, 1])

In [ ]:
random_n = np.random.randint(0, data.shape[0])
random_n = -1


plot_compare_2d(
    true_d=mult_p_true[random_n, 1],
    pred_d=mult_p_pred[random_n, 1],
    savep=ABSOLUTE_PATH + "/results/reaction_diffusion/diffusion.pdf",
)